<a href="https://colab.research.google.com/github/kodikarthik21/CS6910---Fundamentals-of-Deep-Learning/blob/main/Assignment01/CS6910_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb


     |████████████████████████████████| 2.0MB 13.9MB/s 
     |████████████████████████████████| 163kB 58.7MB/s 
     |████████████████████████████████| 102kB 2.1MB/s 
     |████████████████████████████████| 133kB 38.2MB/s 
     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=7cdfd1f54ad7235259947e3148954e0697827dac2e3bfcb3d937d8671524b1e9
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=939087feff94fc9876888cfc3fbda44287a9af6e2d6b635aa468c6162776eb19
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
from keras.datasets import fashion_mnist
from matplotlib import pyplot
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
for i in range(10):
   wandb.init(project="test-drive", reinit=True)
   for j in range(30):
     if y_train[j] == i:
       pyplot.subplot(2,5,(i+1))
       wandb.log({"examples": [wandb.Image(X_train[j], caption="Label")]})
     break
pyplot.show()


_runtime,2
_timestamp,1614759042
_step,0


_runtime,▁
_timestamp,▁
_step,▁
